In [81]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
from imblearn.over_sampling import RandomOverSampler
tf.random.set_seed(1)



import pandas as pd
X_train = pd.read_csv('https://machine-failure-data-20230822-craiguo.s3.us-west-2.amazonaws.com/train.csv')
X_test = pd.read_csv('https://machine-failure-data-20230822-craiguo.s3.us-west-2.amazonaws.com/test.csv')

In [82]:
oversample = RandomOverSampler(random_state=1)

In [83]:
y = X_train['Machine failure']
X_train.drop(columns=['Machine failure'], inplace = True)

In [84]:
X_train, y = oversample.fit_resample(X_train, y)

In [85]:
X_train

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
268557,2973,M19279,M,302.6,310.2,1382,53.6,79,0,1,0,0,0
268558,111380,L51572,L,302.2,310.1,1458,45.1,92,0,0,0,0,0
268559,64996,L47366,L,298.0,308.3,1373,62.4,63,0,0,1,0,0
268560,124990,L55379,L,299.2,310.7,1617,27.0,225,0,0,0,0,0


In [86]:
X_train.tail()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
268557,2973,M19279,M,302.6,310.2,1382,53.6,79,0,1,0,0,0
268558,111380,L51572,L,302.2,310.1,1458,45.1,92,0,0,0,0,0
268559,64996,L47366,L,298.0,308.3,1373,62.4,63,0,0,1,0,0
268560,124990,L55379,L,299.2,310.7,1617,27.0,225,0,0,0,0,0
268561,102976,L49305,L,299.3,308.8,1258,69.4,119,0,0,1,0,0


In [87]:
X_train.nunique()

id                         136429
Product ID                   9976
Type                            3
Air temperature [K]            95
Process temperature [K]        81
Rotational speed [rpm]        952
Torque [Nm]                   611
Tool wear [min]               246
TWF                             2
HDF                             2
PWF                             2
OSF                             2
RNF                             2
dtype: int64

In [88]:
X_test.nunique()

id                         90954
Product ID                  9909
Type                           3
Air temperature [K]           92
Process temperature [K]       84
Rotational speed [rpm]       946
Torque [Nm]                  595
Tool wear [min]              246
TWF                            2
HDF                            2
PWF                            2
OSF                            2
RNF                            2
dtype: int64

In [89]:
# X_combine = pd.concat([X_train, X_test])
# dummy_df = pd.get_dummies(X_combine)
# i = X_train.shape[0]
# dummy_train = dummy_df.iloc[0:i]
# dummy_test = dummy_df.iloc[i:]
X_dropped = X_train[['Product ID','Type']]
X_train.drop(columns=['Product ID','Type'], inplace=True)
X_test.drop(columns=['Product ID','Type'], inplace=True)
# X_train.dtypes

In [90]:
# n = len(X_train.columns) + 2

In [91]:
scaler = StandardScaler()

# X_Scaler = scaler.fit(dummy_train)

# X_train_scaled = X_Scaler.transform(dummy_train)
# X_test_scaled = X_Scaler.transform(dummy_test)

X_Scaler = scaler.fit(X_train)

X_train_scaled = X_Scaler.transform(X_train)
X_test_scaled = X_Scaler.transform(X_test)

In [92]:
# X_train = np.concatenate((X_train_scaled, X_dropped), axis = 1)

In [93]:
n = len(X_train.columns)

nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units = 10, activation = 'relu', input_dim = n))

nn.add(tf.keras.layers.Dense(units = 3, activation = 'relu'))

nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 10)                120       
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 33        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 4         
Total params: 157
Trainable params: 157
Non-trainable params: 0
_________________________________________________________________


In [94]:
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [78]:
y.value_counts()
134281/(134281+2148)

0.9842555468412141

In [95]:
# X_temp = np.asarray(dummy_train).astype(np.float32)
nn.fit(X_train_scaled, y, epochs = 100)

Epoch 1/100
8393/8393 [==============================] - 4s 414us/step - loss: 0.2580 - accuracy: 0.9025
Epoch 2/100
8393/8393 [==============================] - 3s 396us/step - loss: 0.2185 - accuracy: 0.9133
Epoch 3/100
8393/8393 [==============================] - 3s 393us/step - loss: 0.2156 - accuracy: 0.9139
Epoch 4/100
8393/8393 [==============================] - 3s 389us/step - loss: 0.2138 - accuracy: 0.9135
Epoch 5/100
8393/8393 [==============================] - 3s 391us/step - loss: 0.2130 - accuracy: 0.9134
Epoch 6/100
8393/8393 [==============================] - 3s 407us/step - loss: 0.2125 - accuracy: 0.9138
Epoch 7/100
8393/8393 [==============================] - 3s 391us/step - loss: 0.2124 - accuracy: 0.9142
Epoch 8/100
8393/8393 [==============================] - 3s 390us/step - loss: 0.2121 - accuracy: 0.9147
Epoch 9/100
8393/8393 [==============================] - 3s 390us/step - loss: 0.2119 - accuracy: 0.9145
Epoch 10/100
8393/8393 [==============================]

In [80]:
# model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")